In [16]:
# Import statements
import os
import random
import tensorflow as tf
import deeplake
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import PIL.Image
import time
import functools

plt.rcParams['figure.figsize'] = (12, 12)
plt.rcParams['axes.grid'] = False

In [2]:
# Load in style images dataset
ds = deeplake.load('hub://activeloop/wiki-art')

/

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/wiki-art



\

hub://activeloop/wiki-art loaded successfully.



In [3]:
# Mapping for style labels
style_mapping = {
    0: "abstract_expressionism",
    1: "action_painting",
    2: "analytical_cubism",
    3: "art_nouveau_modern",
    4: "baroque",
    5: "color_field_painting",
    6: "contemporary_realism",
    7: "cubism",
    8: "early_renaissance",
    9: "expressionism",
    10: "fauvism",
    11: "high_renaissance",
    12: "impressionism",
    13: "mannerism_late_renaissance",
    14: "minimalism",
    15: "naive_art_primitivism",
    16: "new_realism",
    17: "northern_renaissance",
    18: "pointillism",
    19: "pop_art",
    20: "post_impressionism",
    21: "realism",
    22: "rococo",
    23: "romanticism",
    24: "symbolism",
    25: "synthetic_cubism",
    26: "ukiyo_e"
}

output_base_folder = "./wiki_art_images"
images_per_style = 50
# Create folder for each style
for style_label, style_name in style_mapping.items():
    style_folder = os.path.join(output_base_folder, style_name)
    os.makedirs(style_folder, exist_ok=True)
    
    #Filter dataset
    indices = [i for i, label in enumerate(ds['labels']) if label.numpy() == style_label]
    # Randomly select images of each type
    random_indices = random.sample(indices, min(len(indices), images_per_style))
    
    for i, idx in enumerate(random_indices):
        image_array = ds['images'][idx].numpy()
        
        image = Image.fromarray((image_array * 255).astype(np.uint8))

        image.save(os.path.join(style_folder, f"{style_name}_{i+1}.jpg"))

In [4]:
# Function to convert tensor to image
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [5]:
# Function to laod an image
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

In [6]:
# Function for pretrained VGG19 model
def vgg_layers(layer_names):
    """ Creates a VGG model that returns a list of intermediate output values."""
    # Load our model. Load pretrained VGG, trained on ImageNet data
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
  
    outputs = [vgg.get_layer(name).output for name in layer_names]

    model = tf.keras.Model([vgg.input], outputs)
    return model

In [7]:
# Style content model
class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        "Expects float input in [0,1]"
        inputs = inputs*255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])

        style_outputs = [gram_matrix(style_output) 
                         for style_output in style_outputs]

        content_dict = {content_name: value
                        for content_name, value
                        in zip(self.content_layers, content_outputs)}

        style_dict = {style_name: value
                      for style_name, value
                      in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}

In [8]:
# Function to calculate gram matrix
def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

In [9]:
# Function for loss calculation
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name] - style_targets[name])**2) for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name] - content_targets[name])**2) for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    return style_loss + content_loss

In [10]:
# Function to clip image values to a valid range
def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [11]:
# Training step function
@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_loss(outputs)
        loss += total_variation_weight*tf.image.total_variation(image)

    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

In [12]:
# Style transfer parameters
content_layers = ['block5_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

style_weight=1e-2
content_weight=1e4
total_variation_weight=30
epochs = 2
steps_per_epoch = 100

In [13]:
# Intializing the model
extractor = StyleContentModel(style_layers, content_layers)
opt = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

W0000 00:00:1732291918.311350  209843 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [14]:
content_images_folder = "./Images"
style_images_base_folder = "./wiki_art_images"

content_image_paths = [os.path.join(content_images_folder, fname)
                       for fname in os.listdir(content_images_folder)
                       if fname.lower().endswith(('png', 'jpg', 'jpeg'))]
style_types = [d for d in os.listdir(style_images_base_folder) if os.path.isdir(os.path.join(style_images_base_folder, d))]

In [ ]:
output_base_folder = "./style_images"
for content_path in content_image_paths:
    content_name = os.path.splitext(os.path.basename(content_path))[0]
    content_output_folder = os.path.join(output_base_folder, content_name)
    os.makedirs(content_output_folder, exist_ok=True)
    
    content_image = load_img(content_path)
    
    step = 0
    for style_type in style_types:
        style_type_folder = os.path.join(content_output_folder, style_type)
        os.makedirs(style_type_folder, exist_ok=True)

        style_images = [
            os.path.join(style_images_base_folder, style_type, fname)
            for fname in os.listdir(os.path.join(style_images_base_folder, style_type))
            if fname.lower().endswith(('png', 'jpg', 'jpeg'))
        ]

        for i, style_path in enumerate(style_images):
            style_image = load_img(style_path)

            # Perform style transfer
            style_targets = extractor(style_image)['style']
            content_targets = extractor(content_image)['content']
            stylized_image = tf.Variable(content_image)

            # Training loop
            for n in range(epochs):  # Adjust epochs and steps_per_epoch
                for m in range(steps_per_epoch):
                    train_step(stylized_image)

            # Save the stylized image
            output_image = tensor_to_image(stylized_image)
            output_image.save(os.path.join(style_type_folder, f"{style_type}_{i+1}.jpg"))